In [ ]:
import torch
import os
from interfile import Interfile as Interfile
from PETLibs import *
import PETLibs
import numpy as np
from argparse import ArgumentParser
from torch.nn import functional as F
from matplotlib import pyplot as plt
from tqdm.autonotebook import tqdm
import matplotlib.pyplot as plt
from proj_utils import preprocess_data, postprocess_data, GPU_log_likelihood, GPUBiographProxRecons, GPUBiographEMRecons
from IR_utils import prox_TV_L1
print("hello")


In [1]:
!git clone https://github.com/marsvn/TP_janv25.git

Cloning into 'TP_janv25'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 49 (delta 6), reused 0 (delta 0), pack-reused 37 (from 1)
Receiving objects: 100% (49/49), 68.83 MiB | 12.21 MiB/s, done.
Resolving deltas: 100% (12/12), done.
Updating files: 100% (22/22), done.


In [ ]:
!unzip TP_janv25/lib/interfile.zip -d /content

In [ ]:
!pip install -e /content/interfile

In [ ]:
!python --version

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
!ls -lt /usr/local/lib/

total 743744
drwxr-xr-x 2 root root     4096 Jan  2 14:25 pkgconfig
-rwxr-xr-x 1 root root 13232872 Jan  2 14:20 libmkl_vml_mc3.so.2
-rwxr-xr-x 1 root root 16093776 Jan  2 14:20 libmkl_vml_avx512.so.2
-rwxr-xr-x 1 root root  8059520 Jan  2 14:20 libmkl_vml_cmpt.so.2
-rwxr-xr-x 1 root root  9414920 Jan  2 14:20 libmkl_vml_def.so.2
-rwxr-xr-x 1 root root 32882696 Jan  2 14:20 libmkl_tbb_thread.so.2
-rwxr-xr-x 1 root root 15470528 Jan  2 14:20 libmkl_vml_avx2.so.2
-rwxr-xr-x 1 root root 23443816 Jan  2 14:20 libmkl_sequential.so.2
-rwxr-xr-x 1 root root 46028896 Jan  2 14:20 libmkl_mc3.so.2
-rwxr-xr-x 1 root root 16389768 Jan  2 14:20 libmkl_rt.so.2
-rwxr-xr-x 1 root root  7620808 Jan  2 14:20 libmkl_scalapack_ilp64.so.2
-rwxr-xr-x 1 root root  7601640 Jan  2 14:20 libmkl_scalapack_lp64.so.2
-rwxr-xr-x 1 root root 38738520 Jan  2 14:20 libmkl_intel_thread.so.2
-rwxr-xr-x 1 root root 18330912 Jan  2 14:20 libmkl_intel_lp64.so.2
-rwxr-xr-x 1 root root 27414688 Jan  2 14:20 libmkl_gnu_thread

In [4]:
!cp -r /content/drive/MyDrive/PETLibs /usr/local/lib/python3.10/dist-packages/

In [4]:
!cp -r /content/drive/MyDrive/interfile /usr/local/lib/python3.10/dist-packages/

In [ ]:
from interfile.interfile import Interfile as Interfile


In [6]:
!unzip TP_janv25/lib/pyPETLibs.zip -d /content

Archive:  TP_janv25/lib/pyPETLibs.zip
   creating: /content/pyPETLibs/
  inflating: /content/pyPETLibs/README.md  
   creating: /content/pyPETLibs/tests/
   creating: /content/pyPETLibs/tests/tests_env/
  inflating: /content/pyPETLibs/tests/tests_env/launch_ncu_after.cmd  
  inflating: /content/pyPETLibs/tests/tests_env/launch_sudo_backproj.cmd  
  inflating: /content/pyPETLibs/tests/tests_env/launch_ncu.cmd  
  inflating: /content/pyPETLibs/tests/tests_env/test_notworking.txt  
  inflating: /content/pyPETLibs/tests/tests_env/test.txt  
  inflating: /content/pyPETLibs/tests/tests_env/launch_ncu_notworking.cmd  
  inflating: /content/pyPETLibs/tests/tests_env/test_after.txt  
  inflating: /content/pyPETLibs/tests/tests_env/launch_ncu_working.cmd  
  inflating: /content/pyPETLibs/tests/tests_env/test_working.txt  
  inflating: /content/pyPETLibs/tests/tests_env/test_backproj.txt  
  inflating: /content/pyPETLibs/tests/tests_env/test_default.txt  
  inflating: /content/pyPETLibs/tests/tes

Set the scanner configuration and reconstruction grid options.

In [14]:
!pip install ninja
!apt-get update
!apt-get install -y libgsl-dev
!pip install --upgrade cython
!pip install -e /content/pyPETLibs

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Fetched 128 kB in 2s (80.0 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Don

In [ ]:
!apt-get update
!apt-get install -y libgsl-dev
!pip install --upgrade cython
!python /content/pyPETLibs/setup.py egg_info

In [17]:
%cd /content/pyPETLibs
!python /content/pyPETLibs/setup.py egg_info

/content/pyPETLibs
cppflags= ['-I/usr/include', '-O3', '-DTIMING']
all_libs= ['-L/usr/lib/x86_64-linux-gnu', '-lgsl', '-lgslcblas', '-lm', '-lnvToolsExt']
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/extension.py:139: UserWarning: Unknown Extension options: 'dlink'
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/setuptools/dist.py:294: InformationOnly: Normalizing '2.1.0.dev' to '2.1.0.dev0'
  self.metadata.version = self._normalize_version(self.metadata.version)
running egg_info
writing PETLibs.egg-info/PKG-INFO
writing dependency_links to PETLibs.egg-info/dependency_links.txt
writing requirements to PETLibs.egg-info/requires.txt
writing top-level names to PETLibs.egg-info/top_level.txt
dependency /usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/arrayobject.h won't be automatically included in the manifest: the path must be relative
dependency /usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/arrayscalars.h won't be automati

In [ ]:
!pip install ninja

In [ ]:
!python setup.py clean
!python setup.py build_ext --inplace
!python setup.py install

cppflags= ['-I/usr/include', '-O3', '-DTIMING']
all_libs= ['-L/usr/lib/x86_64-linux-gnu', '-lgsl', '-lgslcblas', '-lm', '-lnvToolsExt']
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/extension.py:139: UserWarning: Unknown Extension options: 'dlink'
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/setuptools/dist.py:294: InformationOnly: Normalizing '2.1.0.dev' to '2.1.0.dev0'
  self.metadata.version = self._normalize_version(self.metadata.version)
running clean
removing 'build/temp.linux-x86_64-cpython-310' (and everything under it)
cppflags= ['-I/usr/include', '-O3', '-DTIMING']
all_libs= ['-L/usr/lib/x86_64-linux-gnu', '-lgsl', '-lgslcblas', '-lm', '-lnvToolsExt']
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/extension.py:139: UserWarning: Unknown Extension options: 'dlink'
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/setuptools/dist.py:294: InformationOnly: Normalizing '2.1.0.dev' to '2.1.0.dev0'
  self.metadata.version = se

In [ ]:
XYZ=True
dim_vol = [128,128,109] #number of voxels in final numpy image (in XYZ convention if XYZ is true)
vox_size=[2.03642,2.03642,2.027] #voxel size in final numpy image (in XYZ convention if XYZ is true)

ParamsGPUEM=PETLibs.recons.GPUBiographReconsParams(lut_name="PET_SIEMENS_BIOGRAPH6_TRUEPOINT_TRUEV.lut",
                                    vox_size=vox_size,im_dim=dim_vol,nit=8, verbose=False,nsubsets=14,VRing=True,XYZ=XYZ, deviceGPU="cuda:0") #,fwhm_mm = 4, nbsigmas = 5)

In [ ]:
%cd ../TP_janv25

In [ ]:
import torch
import os
from interfile.interfile import Interfile as Interfile
from PETLibs import *
import PETLibs
import numpy as np
from argparse import ArgumentParser
from torch.nn import functional as F
from matplotlib import pyplot as plt
from tqdm.autonotebook import tqdm
import matplotlib.pyplot as plt
from proj_utils import preprocess_data, postprocess_data, GPU_log_likelihood, GPUBiographProxRecons, GPUBiographEMRecons
from IR_utils import prox_TV_L1


Load simulated low dose data

In [ ]:
castor_df_hdr=f"realizationj00414-1.cdh"
castor_df_dir="data_TP/j00414/realizationj00414-1"
dose_reduc = 5.0

castor_df_hdr_fpath=os.path.join(castor_df_dir,castor_df_hdr)
dict_sino=PETLibs.recons.GPURecons.GPUCASToRSinos(castor_df_hdr_fpath, castor_df_dir, reconsParams=ParamsGPUEM)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!cp -r /usr/local/lib/python3.10/dist-packages/PETLibs /content/drive/MyDrive/

In [18]:
import PETLibs

AttributeError: module 'PETLibs.ios' has no attribute 'CASToRInterfile'

In [13]:
!rm -rf /usr/local/lib/python3.10/dist-packages/PETLibs

In [22]:
!more /usr/local/lib/python3.10/dist-packages/PETLibs.egg-link

/content/pyPETLibs
.


In [19]:
!ls -lt /usr/local/lib/python3.10/dist-packages/

total 34572
drwxr-xr-x  1 root root     4096 Jan  4 19:12 nibabel
-rw-r--r--  1 root root       19 Jan  4 19:10 easy-install.pth
-rw-r--r--  1 root root       20 Jan  4 19:10 PETLibs.egg-link
drwxr-xr-x  2 root root     4096 Jan  4 17:57 interfile-0.3.1.dist-info
drwx------  5 root root     4096 Jan  4 17:57 interfile
drwxr-xr-x  2 root root     4096 Jan  4 17:57 simplewrap-0.3.3.dist-info
drwxr-xr-x  5 root root     4096 Jan  4 17:57 simplewrap
drwxr-xr-x  3 root root     4096 Jan  4 17:56 ninja-1.11.1.3.dist-info
drwxr-xr-x  3 root root     4096 Jan  4 17:56 ninja
drwxr-xr-x  1 root root     4096 Jan  4 17:50 zipp
drwxr-xr-x  1 root root     4096 Jan  4 17:50 importlib_metadata
drwxr-xr-x  1 root root     4096 Jan  4 17:50 __pycache__
drwxr-xr-x  1 root root     4096 Jan  4 17:50 debugpy
drwxr-xr-x  1 root root     4096 Jan  4 17:49 huggingface_hub
drwxr-xr-x  1 root root     4096 Jan  4 17:49 pydotplus
drwxr-xr-x  1 root root     4096 Jan  4 17:49 optree
drwxr-xr-x  1 root root     

In [ ]:
!cp -r /content/drive/MyDrive/pyPETLibs /usr/local/lib/python3.10/dist-packages/


In [ ]:
!ls /usr/local/lib/python3.10/dist-packages/

Load corresponding simulated high dose OSEM reconstruction

In [ ]:
ordose_hdr=Interfile.load("data_TP/j00414/originaldose-red-j00414-1-8it-14sub.hdr")
ordose_hdr['name of data file']['value']="originaldose-red-j00414-1-8it-14sub.ima"
ordose,_,_,_,_=PETLibs.ios.getNPArrayFromCASToRInterfileHeader(ordose_hdr,"data_TP/j00414",verbose=False)


xtrue = ordose

**Goal for this lab session**: reconstruction images with a "high-dose iamge quality" from low-dose data.

# Part I: ADMM reconstruction

A common way of reconstructing a 3D image $\overline{x}$ from the set of sinograms $y$ is to estimate $\overline{x}$ by the minimizer $\hat{x}$ of

$$
 L(x) + \lambda g(D x) \qquad  (P)
$$

Usually, $g$ is chosen as an $\ell_1$ norm and $D$ typically refer to a wavelet transform or the gradient operator. In the latter, the regularization is called **total variation**.


For Problem (P), the alternating direction method of multipliers (ADMM) writes as

$x^{(k+1)} = \rm{prox}_{L/\rho}(z^{(k)}+u^{(k)})$

$z^{(k+1)} = \rm{prox}_{\lambda g(D \cdot)/\rho}(x^{(k+1)}  -u^{(k)})$

$u^{(k+1)}=u^{(k)} + x^{(k+1)} -  z^{(k+1)}$

where $\rho>0$.

The sequences of iterates $(x_k)_k$ and $(z_k)_k$ converge to $\hat{x}$.

Below is the template of an implementation of ADMM for solving (P). Complete it and study the impact of $\lambda$ and $\rho$ on image quality.

In [ ]:

# def admm(x0, nit_admm, reg_par, pnlt_beta, BiographParams, dict_sino):

#     v=torch.tensor(np.zeros_like(x0), dtype=torch.float32,device="cuda")
#     u=torch.zeros_like(v)

#     for _ in range(nit_admm):


#         ...
#         x=GPUBiographProxRecons(BiographReconsParams=BiographParams,xprox=...,pnlt_beta=pnlt_beta,dict_sino=dict_sino, tensor_output = True)
#         ...



#     return x



def admm(x0, nit_admm, reg_par, pnlt_beta, BiographParams, dict_sino):

    v=torch.tensor(np.zeros_like(x0), dtype=torch.float32,device="cuda:0")
    u=torch.zeros_like(v, device="cuda:0")

    for _ in range(nit_admm):


        x=GPUBiographProxRecons(BiographReconsParams=BiographParams,xprox=v-u,pnlt_beta=pnlt_beta,dict_sino=dict_sino, tensor_output = True).squeeze()
        n_image=preprocess_data(x+u, tensorFlag=True)
        v = prox_TV_L1(n_image, lamb=reg_par/pnlt_beta, maxit=1000,check=100, verbose=0)
        v=postprocess_data(v, tensorFlag=True)

        u = u + x - v


    return x

In [ ]:
nit_admm = 10
rho = 1e-6
lbd = 0.00002

x_admm = admm(x0=ordose, nit_admm=nit_admm, reg_par=lbd, pnlt_beta=rho, BiographParams=ParamsGPUEM, dict_sino=dict_sino)
x_admm = x_admm.squeeze().cpu().detach().numpy()

fig=PETLibs.utils.display3D_1cbar(x_admm,interpolation='none', vmin = 0, vmax= 2600, title='reconstruction with ADMM - TV regularization')

# Part II: Proximity operator vs denoiser

Reconstruct the low-dose data using OSEM with 8 iterations and 14 subsets.

In [ ]:
ParamsGPUEM=PETLibs.recons.GPUBiographReconsParams(lut_name="PET_SIEMENS_BIOGRAPH6_TRUEPOINT_TRUEV.lut",
                                    vox_size=vox_size,im_dim=dim_vol,nit=10, verbose=False,nsubsets=14,VRing=True,XYZ=XYZ,deviceGPU="cuda:0")


recon_EM_LD = GPUBiographEMRecons(castor_df_hdr_fpath, castor_df_dir,reconsParams=ParamsGPUEM, dict_sino=dict_sino, verbose=False)

Display the high-dose reference and low-dose OSEM reconstruction.

In [ ]:
fig=PETLibs.utils.display3D_1cbar(ordose,interpolation='none', vmin = 0, vmax= 2600, title="high dose reference")
fig=PETLibs.utils.display3D_1cbar(recon_EM_LD,interpolation='none', vmin = 0, vmax= 2600, title="OSEM low dose")

Apply different post-reconstruction denoisers on the OSEM low dose image.

In [ ]:
xnoisy = recon_EM_LD

### Post reconstruction denoiser: TV prox

In [ ]:
from IR_utils import prox_TV_L1

reg_par = 250
x_noisy = preprocess_data(xnoisy)
xtv = prox_TV_L1(x_noisy, lamb=reg_par, maxit=1000)
xtv = postprocess_data(xtv)

### Post reconstruction denoiser: gaussian filtering

In [ ]:
import math
import numbers


class GaussianSmoothing(torch.nn.Module):
    """
    Apply gaussian smoothing on a
    1d, 2d or 3d tensor. Filtering is performed seperately for each channel
    in the input using a depthwise convolution.
    Arguments:
        channels (int, sequence): Number of channels of the input tensors. Output will
            have this number of channels as well.
        kernel_size (int, sequence): Size of the gaussian kernel.
        sigma (float, sequence): Standard deviation of the gaussian kernel.
        dim (int, optional): The number of dimensions of the data.
            Default value is 2 (spatial).
    """
    def __init__(self, channels, kernel_size, sigma, dim=2):
        super(GaussianSmoothing, self).__init__()
        if isinstance(kernel_size, numbers.Number):
            kernel_size = [kernel_size] * dim
        if isinstance(sigma, numbers.Number):
            sigma = [sigma] * dim
            #you should smooth with a Gaussian filter approximately three times the size of your voxel.

        # The gaussian kernel is the product of the
        # gaussian function of each dimension.
        kernel = 1
        meshgrids = torch.meshgrid(
            [
                torch.arange(size, dtype=torch.float32)
                for size in kernel_size
            ]
        )
        for size, std, mgrid in zip(kernel_size, sigma, meshgrids):
            mean = (size - 1) / 2
            kernel *= 1 / (std * math.sqrt(2 * math.pi)) * \
                      torch.exp(-((mgrid - mean) / std) ** 2 / 2)

        # Make sure sum of values in gaussian kernel equals 1.
        kernel = kernel / torch.sum(kernel)

        # Reshape to depthwise convolutional weight
        kernel = kernel.view(1, 1, *kernel.size())
        kernel = kernel.repeat(channels, *[1] * (kernel.dim() - 1))

        self.register_buffer('weight', kernel)
        self.groups = channels

        if dim == 1:
            self.conv = F.conv1d
        elif dim == 2:
            self.conv = F.conv2d
        elif dim == 3:
            self.conv = F.conv3d
        else:
            raise RuntimeError(
                'Only 1, 2 and 3 dimensions are supported. Received {}.'.format(dim)
            )

    def forward(self, input):
        """
        Apply gaussian filter to input.
        Arguments:
            input (torch.Tensor): Input to apply gaussian filter on.
        Returns:
            filtered (torch.Tensor): Filtered output.
        """
        return self.conv(input, weight=self.weight, groups=self.groups, padding=2)



def sigma2fwhm(sigma):
     return sigma * np.sqrt(8 * np.log(2))


def fwhm2sigma(fwhm):
    return fwhm / np.sqrt(8 * np.log(2))

In [ ]:
kernel_size = 5
fwhm = 1.5
sigma_gaussian = fwhm2sigma(fwhm)
x_noisy = preprocess_data(xnoisy)
smoothing = GaussianSmoothing(1, kernel_size, sigma_gaussian, 3).cuda()
xgauss = postprocess_data(smoothing(x_noisy))

### Post reconstruction denoiser: BM3D

In [ ]:
import bm4d
reg_par = 700
xbm3d_2=bm4d.bm4d(xnoisy.squeeze(), reg_par)

### Post reconstruction deep denoisers

In [ ]:
from DL_utils import MyDenoiser, MyLDenoiser

net0 = MyLDenoiser()
net1 = MyDenoiser()
net2 = MyDenoiser()


if torch.cuda.is_available():  # Move to GPU if possible
    net0 = net0.cuda()
    net1 = net1.cuda()
    net2 = net2.cuda()

Loading net1 and net 2

In [ ]:
filename = './net0.ckpt'
checkpoint = torch.load(filename)
net0.load_state_dict(checkpoint['state_dict'], strict=True)
net0.eval()


filename = './net2.ckpt'
checkpoint = torch.load(filename)
net2.load_state_dict(checkpoint['state_dict'], strict=True)
net2.eval()

filename = 'net1.ckpt'
checkpoint = torch.load(filename)
net1.load_state_dict(checkpoint['state_dict'], strict=True)
net1.eval()
net1.cuda()



In [ ]:
x_noisy = preprocess_data(xnoisy)
xnet0 = postprocess_data(net0(x_noisy))
xnet2 = postprocess_data(net2(x_noisy))
xnet1 = postprocess_data(net1(x_noisy))

Comparing all denoisers

In [ ]:
fig=PETLibs.utils.display3D_1cbar(xtrue,interpolation='none', vmin = 0, vmax= 2600, title='Reference')
fig=PETLibs.utils.display3D_1cbar(xnoisy,interpolation='none', vmin = 0, vmax= 2600, title='EM')
fig=PETLibs.utils.display3D_1cbar(xtv,interpolation='none', vmin = 0, vmax= 2600, title='Denoised with total variation')
fig=PETLibs.utils.display3D_1cbar(xgauss,interpolation='none', vmin = 0, vmax= 2600, title='Denoised with Gaussian')
fig=PETLibs.utils.display3D_1cbar(xbm3d_2,interpolation='none', vmin = 0, vmax= 2600, title='Denoised with BM3D')
fig=PETLibs.utils.display3D_1cbar(xnet0,interpolation='none', vmin = 0, vmax= 2600, title='Denoised with net0')
fig=PETLibs.utils.display3D_1cbar(xnet1,interpolation='none', vmin = 0, vmax= 2600, title='Denoised with net1')
fig=PETLibs.utils.display3D_1cbar(xnet2,interpolation='none', vmin = 0, vmax= 2600, title='Denoised with net2')

TODO: add quantitative metrics (e.g. MSE, MAE, SSIM) using

In [ ]:
import torchmetrics
compute_MSE = torch.nn.MSELoss()
compute_MAE = torchmetrics.MeanAbsoluteError()
compute_SS = torchmetrics.StructuralSimilarityIndexMeasure()

In [ ]:
# comparing the MSE associated with each method
criterion = torch.nn.MSELoss()
mse_em = criterion(torch.tensor(xnoisy), torch.tensor(xtrue)).item()
mse_tv = criterion(torch.tensor(xtv), torch.tensor(xtrue)).item()
mse_bm3d = criterion(torch.tensor(xbm3d_2), torch.tensor(xtrue)).item()
mse_gauss = criterion(torch.tensor(xgauss), torch.tensor(xtrue)).item()
mse_net0 = criterion(torch.tensor(xnet0), torch.tensor(xtrue)).item()
mse_net2 = criterion(torch.tensor(xnet2), torch.tensor(xtrue)).item()
mse_net1 = criterion(torch.tensor(xnet1), torch.tensor(xtrue)).item()



print('MSE EM: ', mse_em)
print('MSE TV: ', mse_tv)
print('MSE BM3D: ', mse_bm3d)
print('MSE Gauss: ', mse_gauss)
print('MSE net0: ', mse_net0)
print('MSE net1: ', mse_net1)
print('MSE net2: ', mse_net2)


TODO: measure execution time of all denoisers using

In [ ]:
import timeit

# Part III: PnP ADMM with prox surrogate

Edit admm to replace the proximity operator of TV with a deep model (net1 or net2).

In [ ]:
def compute_norm_jac(x, y, tol, max_iter):
    '''
    Perform evaluation of the spectral radius of the jacobian dy/dx(x)

    Arguments:
        - x: input where the Jacobian is computed (torch.Tensor).
        - y: output of the operator of which the spectral radius of the Jacobian is computed  (torch.Tensor).

    Returns:
        spectral_radius: last evaluation of spectral radius of Jacobian.
        lst_spectral_radius: list of values of spectral radius of Jacobian across iterations.
    '''
    torch.random.manual_seed(0)
    u = torch.randn_like(x)
    u = u/torch.matmul(u.reshape(u.shape[0], 1, -1), u.reshape(u.shape[0], -1, 1)).view(u.shape[0], 1, 1, 1)

    z_old = torch.zeros(u.shape[0])
    with tqdm(total=max_iter, desc="Power Method",leave=False) as pbar_jac:
        for it in range(max_iter-1):
            w = torch.ones_like(y, requires_grad=True)  # Double backward trick. From https://gist.github.com/apaszke/c7257ac04cb8debb82221764f6d117ad
            v = torch.autograd.grad(torch.autograd.grad(y, x, w, create_graph=True), w, u, create_graph=False)[0]  # Ju
            w.detach_()
            v.detach_()
            v = torch.autograd.grad(y, x, v, retain_graph=True, create_graph=False)[0]  # vtJt

            z = torch.matmul(u.reshape(u.shape[0], 1, -1), v.reshape(v.shape[0], -1, 1)) / torch.matmul(u.reshape(u.shape[0], 1, -1), u.reshape(u.shape[0], -1, 1))
            if it > 0:
                rel_var = torch.norm(z-z_old)
                if rel_var < tol:
                    break
            z_old = z.clone()

            u = v/torch.matmul(v.reshape(v.shape[0], 1, -1), v.reshape(v.shape[0], -1, 1)).view(v.shape[0], 1, 1, 1)
            pbar_jac.update(1)
            u.detach_()


        pbar_jac.update(1)
        w.detach_()
        v.detach_()
    return z.view(-1).sqrt()

def pnp_admm(x0, nit_admm, model, pnlt_beta, max_iter_jacobian, prec, device, BiographParams, dict_sino, target, nb_everyIter_jac = 1):

    v=torch.tensor(np.zeros_like(x0), dtype=torch.float32,device="cuda")
    u=torch.zeros_like(v)

    criterion = torch.nn.MSELoss()

    mse = []
    log_l = []
    norm1 = []
    norm2 = []
    reg = []
    model_run=lambda v: model(v)[0]


    for i in range(nit_admm):


        # xprox=torch.swapaxes(v-u,2,0)
        x=GPUBiographProxRecons(BiographReconsParams=BiographParams,xprox=v-u,pnlt_beta=pnlt_beta,dict_sino=dict_sino, tensor_output = True).squeeze()

        # x_ZYX=torch.swapaxes(x,2,0)
        loss = criterion(preprocess_data(x, tensorFlag=True), target)
        mse.append(loss.item())
        l = GPU_log_likelihood(x, eps =0,BiographParams=BiographParams,dict_sino=dict_sino)
        log_l.append(l)

        v_old = v
        n_image=preprocess_data(x+u, tensorFlag=True)
        v = model_run(n_image)
        v=postprocess_data(v.squeeze(), tensorFlag=True)

        out_detached = n_image.detach()
        out_detached.requires_grad_()
        out_net_reg = 2.*model_run(out_detached)-out_detached
        if i%nb_everyIter_jac==0:
            reg_loss = compute_norm_jac(out_detached, out_net_reg, tol = 1e-3, max_iter = max_iter_jacobian)
            reg_loss = reg_loss.item()
            print("jac=", reg_loss)
            reg.append(reg_loss)


        primalN=torch.linalg.norm(x-v)
        dualN=torch.linalg.norm(v-v_old)

        if i > 0:
            if (dualN/torch.linalg.norm(v)<prec) & (primalN/torch.linalg.norm(v)<prec):
                print("Convergence reached")
                break

            norm1.append(primalN.item())
            norm2.append(dualN.item())

        u = u + x - v

    results={}
    results["x"]=x
    results["mse"]=mse
    results["reg"]=reg
    results["logl"]=log_l
    results["norm1"]=norm1
    results["norm2"]=norm2

    return results


Setting hyperparameters of pnp-admm

In [ ]:
max_iter_jacobian = 3
device = 'cuda'
prec = 1e-3
nit_admm = 40
pnlt_beta = 1e-7
nb_everyIter_jac = 10

In [ ]:
results_admm = pnp_admm(x0=xnoisy, nit_admm=nit_admm, model=net1, pnlt_beta=pnlt_beta, max_iter_jacobian=max_iter_jacobian, prec=prec, device=device, BiographParams=ParamsGPUEM, dict_sino=dict_sino, target=preprocess_data(ordose), nb_everyIter_jac=nb_everyIter_jac)
x_admm = results_admm["x"].squeeze().cpu().detach().numpy()
mse_admm = results_admm["mse"]
reg_admm = results_admm["reg"]
logl_admm = results_admm["logl"]
norm1_admm = results_admm["norm1"]
norm2_admm = results_admm["norm2"]


fig=PETLibs.utils.display3D_1cbar(x_admm,interpolation='none', vmin=0, vmax=2600, title='reconstruction with PnP ADMM - net3')

#plotting mse as a function of iteration
plt.figure()
plt.plot(mse_admm)
plt.title('MSE')
plt.show()

#plotting log likelihood as a function of iteration
plt.figure()
plt.plot(logl_admm)
plt.title('Log likelihood')
plt.show()

#plotting norm1 as a function of iteration
plt.figure()
plt.plot(norm1_admm)
plt.title('Norm1')
plt.show()

#plotting norm2 as a function of iteration
plt.figure()
plt.plot(norm2_admm)
plt.title('Norm2')
plt.show()

#plotting reg as a function of iteration
plt.figure()
plt.plot(reg_admm)
plt.title('Reg')
plt.show()


In [ ]:

results_admm2 = pnp_admm(x0=xnoisy, nit_admm=nit_admm, model=net2, pnlt_beta=pnlt_beta, max_iter_jacobian=max_iter_jacobian, prec=prec, device=device, BiographParams=ParamsGPUEM, dict_sino=dict_sino, target=preprocess_data(ordose),nb_everyIter_jac=nb_everyIter_jac)

x_admm2 = results_admm2["x"].squeeze().cpu().detach().numpy()
mse_admm2 = results_admm2["mse"]
reg_admm2 = results_admm2["reg"]
logl_admm2 = results_admm2["logl"]
norm1_admm2 = results_admm2["norm1"]
norm2_admm2 = results_admm2["norm2"]


fig=PETLibs.utils.display3D_1cbar(x_admm2,interpolation='none', vmin=0, vmax=2600, title='reconstruction with PnP ADMM - net2')

#plotting mse as a function of iteration
plt.figure()
plt.plot(mse_admm2)
plt.title('MSE')
plt.show()

#plotting log likelihood as a function of iteration
plt.figure()
plt.plot(logl_admm2)
plt.title('Log likelihood')
plt.show()

#plotting norm1 as a function of iteration
plt.figure()
plt.plot(norm1_admm2)
plt.title('Norm1')
plt.show()

#plotting norm2 as a function of iteration
plt.figure()
plt.plot(norm2_admm2)
plt.title('Norm2')
plt.show()

#plotting reg as a function of iteration
plt.figure()
plt.plot(reg_admm2)
plt.title('Reg')
plt.show()


The iterates of PnP ADMM converge when the denoiser $D$ is firmly nonexpansive i.e.

            2D - I is 1-lipschitz (Lip)

            
Given that an operator $A$ is L-lipchitz if the spectral norm of its jacobian (or gradient for functions $\mathbb{R}^N \mapsto \mathbb{R}$) is smaller than L.
Use the power method (https://math.univ-cotedazur.fr/~frapetti/CorsoF/cours4part2.pdf) to compute the spectral norm of the jacobian of $2D - I$ (denoted $J$).

Hint 1: the power method is applicable to symmetric operators so you should use that $||J^T J|| = ||J||^2$.

Hint 2: Double backward trick. From https://gist.github.com/apaszke/c7257ac04cb8debb82221764f6d117ad

In [ ]:
def compute_norm_jac(x, y, tol, max_iter):
    '''
    Perform evaluation of the spectral radius of the jacobian dy/dx(x)

    Arguments:
        - x: input where the Jacobian is computed (torch.Tensor).
        - y: output of the operator of which the spectral radius of the Jacobian is computed  (torch.Tensor).
    '''
    torch.random.manual_seed(0)
    u = torch.randn_like(x)
    u = u/torch.matmul(u.reshape(u.shape[0], 1, -1), u.reshape(u.shape[0], -1, 1)).view(u.shape[0], 1, 1, 1)

    z_old = torch.zeros(u.shape[0])
    with tqdm(total=max_iter, desc="Power Method",leave=False) as pbar_jac:
        for it in range(max_iter-1):
            w = torch.ones_like(y, requires_grad=True)  # Double backward trick. From https://gist.github.com/apaszke/c7257ac04cb8debb82221764f6d117ad
            v = torch.autograd.grad(torch.autograd.grad(y, x, w, create_graph=True), w, u, create_graph=False)[0]  # Ju
            w.detach_()
            v.detach_()
            v = torch.autograd.grad(y, x, v, retain_graph=True, create_graph=False)[0]  # vtJt

            z = torch.matmul(u.reshape(u.shape[0], 1, -1), v.reshape(v.shape[0], -1, 1)) / torch.matmul(u.reshape(u.shape[0], 1, -1), u.reshape(u.shape[0], -1, 1))
            if it > 0:
                rel_var = torch.norm(z-z_old)
                if rel_var < tol:
                    break
            z_old = z.clone()

            u = v/torch.matmul(v.reshape(v.shape[0], 1, -1), v.reshape(v.shape[0], -1, 1)).view(v.shape[0], 1, 1, 1)
            pbar_jac.update(1)
            u.detach_()


        pbar_jac.update(1)
        w.detach_()
        v.detach_()
    return z.view(-1).sqrt()


def func_unknown(y, xref, model, max_iter, eta, lr) :
    '''
    ?
    '''
    torch.random.manual_seed(0)
    n = torch.rand_like(xref)
    n = n/torch.norm(n)*eta
    n.requires_grad = True
    optimizer = torch.optim.Adam([n], lr=lr)
    for i in range(max_iter):
        optimizer.zero_grad()
        loss = -torch.norm(y.detach() - model(xref.detach() + n))
        # print("loss", loss.item())
        loss.backward()
        optimizer.step()
        n.data = n.data/torch.norm(n.data)*eta

    return (xref + n).detach(), torch.norm(y.detach() - model(xref.detach() + n)).item()/eta



In [ ]:
x_noisy = preprocess_data(xnoisy).detach().requires_grad_()
e = net2(x_noisy)-x_noisy

Test whether net2 or net3 satisfies the FNE condition on the OSEM image.

In [ ]:
del x_noisy
x_noisy = preprocess_data(xnoisy).detach().requires_grad_()
x_noisy = x_noisy
_2Dx_x = 2*net2(x_noisy) - x_noisy
jac_norm_net2 = compute_norm_jac(x_noisy, _2Dx_x, tol = 1e-3, max_iter = 10).item()
print("jac net 2 : ",jac_norm_net2)

_2Dx_x = 2*net1(x_noisy) - x_noisy
jac_norm_net1 = compute_norm_jac(x_noisy, _2Dx_x, tol = 1e-3, max_iter = 10).item()
print("jac net 1 : ",jac_norm_net1)

What is the following function doing?

In [ ]:
def func_unknown(y, xref, model, max_iter, eta, lr) :
    '''
    ?
    '''
    torch.random.manual_seed(0)
    n = torch.rand_like(xref)
    n = n/torch.norm(n)*eta
    n.requires_grad = True
    optimizer = torch.optim.Adam([n], lr=lr)
    for i in range(max_iter):
        optimizer.zero_grad()
        loss = -torch.norm(y.detach() - model(xref.detach() + n))
        # print("loss", loss.item())
        loss.backward()
        optimizer.step()
        n.data = n.data/torch.norm(n.data)*eta

    return (xref + n).detach(), torch.norm(y.detach() - model(xref.detach() + n)).item()/eta

In [ ]:
x_noisy = preprocess_data(xnoisy).detach().requires_grad_()

x_per, loss = func_unknown(net2(x_noisy), x_noisy, net2, 1000, 2000, 0.002)
print(loss)
fig=PETLibs.utils.display3D_1cbar(postprocess_data(x_per),interpolation='none', perc=0.7)
fig=PETLibs.utils.display3D_1cbar(postprocess_data(x_noisy),interpolation='none', perc=0.7)
fig=PETLibs.utils.display3D_1cbar(postprocess_data(net2(x_noisy)),interpolation='none', perc=0.7)
fig=PETLibs.utils.display3D_1cbar(postprocess_data(net2(x_per)),interpolation='none', perc=0.7)
fig=PETLibs.utils.display3D_1cbar(postprocess_data(net2(x_per)-net2(x_noisy)),interpolation='none', perc=0.7)
fig=PETLibs.utils.display3D_1cbar(postprocess_data(x_per-x_noisy),interpolation='none', perc=0.7)

In [ ]:
x_noisy = preprocess_data(xnoisy).detach().requires_grad_()

x_per, loss = func_unknown(net1(x_noisy), x_noisy , net1 , 1000, 2000, 0.002)
print(loss)

fig=PETLibs.utils.display3D_1cbar(postprocess_data(x_per),interpolation='none', perc=0.7)
fig=PETLibs.utils.display3D_1cbar(postprocess_data(x_noisy),interpolation='none', perc=0.7)
fig=PETLibs.utils.display3D_1cbar(postprocess_data(net1(x_noisy)),interpolation='none', perc=0.7)
fig=PETLibs.utils.display3D_1cbar(postprocess_data(net1(x_per)),interpolation='none', perc=0.7)
fig=PETLibs.utils.display3D_1cbar(postprocess_data(net1(x_per)-net1(x_noisy)),interpolation='none', perc=0.7)
fig=PETLibs.utils.display3D_1cbar(postprocess_data(x_per-x_noisy),interpolation='none', perc=0.7)


### Making an arbitrary denoiser a FNE denoiser

In [ ]:
Lip = 1.0 #replace by jac_norm_net2 or jac_norm_net3
net = torch.nn.Identity() #replace by net2 or net3
FNE_net = lambda x: (1.0-0.5/Lip)*x+0.5/Lip*net(x)

In [ ]:
results_admm_FNE = pnp_admm(x0=xnoisy, nit_admm=30, model=FNE_net, pnlt_beta=pnlt_beta, max_iter_jacobian=2, prec=prec, device=device, BiographParams=ParamsGPUEM, dict_sino=dict_sino, target=preprocess_data(ordose))

In [ ]:
x_admm_FNE = results_admm_FNE["x"].squeeze().cpu().detach().numpy()
mse_admm_FNE = results_admm_FNE["mse"]
# reg_admm_jac = results_admm_FNE["reg"]
logl_admm_FNE = results_admm_FNE["logl"]
norm1_admm_FNE = results_admm_FNE["norm1"]
norm2_admm_FNE = results_admm_FNE["norm2"]

fig=PETLibs.utils.display3D_1cbar(x_admm_FNE,interpolation='none', perc=0.7, title='reconstruction with PnP ADMM FNE')

## Generalization of previous results to patient with Alzheimer disease

In [ ]:

castor_df_hdr=f"realizationj00366-1.cdh"
castor_df_dir="data_TP/j00366/realizationj00366-1"

castor_df_hdr_fpath=os.path.join(castor_df_dir,castor_df_hdr)
dict_sino=PETLibs.recons.GPURecons.GPUCASToRSinos(castor_df_hdr_fpath, castor_df_dir, reconsParams=ParamsGPUEM)

ParamsGPUEM=PETLibs.recons.GPUBiographReconsParams(lut_name="PET_SIEMENS_BIOGRAPH6_TRUEPOINT_TRUEV.lut",
                                    vox_size=vox_size,im_dim=dim_vol,nit=10, verbose=False,nsubsets=14,VRing=True,XYZ=XYZ)


recon_EM_LD = PETLibs.recons.GPUEMRecons(castor_df_hdr_fpath, castor_df_dir,reconsParams=ParamsGPUEM, dict_sino=dict_sino, verbose=False)

ordose_hdr=Interfile.load("data_TP/j00366/originaldose-red-j00366-1-10it-14sub.hdr")
ordose_hdr['name of data file']['value']="originaldose-red-j00366-1-10it-14sub.ima"
ordose,_,_,_,_=PETLibs.ios.getNPArrayFromCASToRInterfileHeader(ordose_hdr,"data_TP/j00366",verbose=False)


x_noisy = preprocess_data(recon_EM_LD)


# Part IV: PnP FB with gradient surrogate

GS-net1 is a denoiser that writes as $N(x) + J_{N(x)}^\top (x - N(x))$.

The forward backward algorithm for PET reconstruction (minimize $LL(x) + r(x)$) with $r$ non convex function with lipschitz differentiable gradient writes as
$$
{x^{k+1}} {= prox_{\tau LL+ \iota_{\left[0,+\infty\right[^N}}(x^k -  \tau \nabla r(x^k))}\\
%x^{k+1}_\lambda = prox_{\tau LL+ \iota_{\left[0,+\infty\right[^N}}(\lambda \tau {D_{\theta}}(x^k_\lambda) + (1-\lambda \tau) x^k_\lambda)$$
for $\tau < 2/L$

When $L$ is unknown, a backtracking linesearch can be used to compute the step size $\tau$.

Explore the forward backward algorithm when $r = \|\nabla x\|^2_2$ as implemented below. What could be a PnP version of the FB algorithm where GS-net1 acts as a regularization ?

In [ ]:

def fb(x0, nit_fb, reg_par, BiographParams, dict_sino):

    x=torch.tensor(np.zeros_like(x0), dtype=torch.float32,device="cuda")

    # define the finite differences operator

    Dh = torch.tensor([[[0., 0., 0.],
                        [0., 0., 0.],
                        [0., 0., 0.]],
                        [[0., 0., 0.],
                        [-1, 1., 0.],
                        [0., 0., 0.]],
                        [[0., 0., 0.],
                        [0., 0., 0.],
                        [0., 0., 0.]]]).view(1,1,3,3,3).cuda()


    Dp = torch.tensor([[[0., 0., 0.],
                    [0., -1., 0.],
                    [0., 0., 0.]],
                    [[0., 0., 0.],
                    [0., 1., 0.],
                    [0., 0., 0.]],
                    [[0., 0., 0.],
                    [0., 0., 0.],
                    [0., 0., 0.]]]).view(1,1,3,3,3).cuda()

    Dv = torch.tensor([[[0., 0., 0.],
                    [0., 0., 0.],
                    [0., 0., 0.]],
                    [[0., -1., 0.],
                    [0., 1., 0.],
                    [0., 0., 0.]],
                    [[0., 0., 0.],
                    [0., 0., 0.],
                    [0., 0., 0.]]]).view(1,1,3,3,3).cuda()


    D = torch.concatenate([Dh, Dp, Dv], dim=0)

    # and its adjoint

    Dh_star = torch.tensor([[[0., 0., 0.],
                        [0., 0., 0.],
                        [0., 0., 0.]],
                        [[0., 0., 0.],
                        [0., 1., -1.],
                        [0., 0., 0.]],
                        [[0., 0., 0.],
                        [0., 0., 0.],
                        [0., 0., 0.]]]).view(1,1,3,3,3).cuda()


    Dp_star = torch.tensor([[[0., 0., 0.],
                    [0., 0., 0.],
                    [0., 0., 0.]],
                    [[0., 0., 0.],
                    [0., 1., 0.],
                    [0., 0., 0.]],
                    [[0., 0., 0.],
                    [0., -1., 0.],
                    [0., 0., 0.]]]).view(1,1,3,3,3).cuda()

    Dv_star = torch.tensor([[[0., 0., 0.],
                    [0., 0., 0.],
                    [0., 0., 0.]],
                    [[0., 0., 0.],
                    [0., 1., 0.],
                    [0., -1., 0.]],
                    [[0., 0., 0.],
                    [0., 0., 0.],
                    [0., 0., 0.]]]).view(1,1,3,3,3).cuda()


    D_star = torch.concatenate([Dh_star, Dp_star, Dv_star], dim=1)

    tau = 0.1/(reg_par)

    # parameters for backtracking linesearch
    epsilon = 1e-9
    delta = epsilon + 1e-6
    gamma = 1e-5
    eta = 0.9
    i=0
    xprec = x


    while i <nit_fb and delta > epsilon:

        xprec = x
        backtracking_check = True
        sub_counter = 0


        while backtracking_check:

            u = x - (tau*reg_par*postprocess_data(F.conv3d(F.conv3d(preprocess_data(x, tensorFlag=True), D, padding=1), D_star, padding=1), tensorFlag=True))
            # xprox=torch.swapaxes(u,2,0)
            xprox = u
            x=GPUBiographProxRecons(BiographReconsParams=BiographParams,xprox=xprox,pnlt_beta=1.0/tau,dict_sino=dict_sino, tensor_output = True).squeeze()

            lk = GPU_log_likelihood(x, eps =0, BiographParams=BiographParams,dict_sino=dict_sino)
            # x=torch.swapaxes(x,2,0)

            gx = reg_par*0.5*torch.norm(F.conv3d(preprocess_data(x, tensorFlag=True), D, padding=1))**2

            if sub_counter > 0:
                lossprec = loss
            if i== 0 and sub_counter == 0:
                linit = lk

            loss = -lk + lbd*gx

            ## Backtracking linesearch: If loss function is not increasing, we decrease tau
            if (sub_counter == 0 or torch.abs(lossprec -loss) < (gamma/tau)*torch.norm(xprec-x)**2) :
                # continue backtracking
                tau = eta*tau
                x = xprec
                backtracking_check = True
            else:
                # move on to next iteration
                if sub_counter > 0 and i > 0:
                    delta  = torch.abs((lossprec-loss)/linit)
                backtracking_check = False


            sub_counter += 1
        i += 1



    return x

In [ ]:
nit_fb = 20
lbd = 0.0000002


x_admm = fb(x0=ordose, nit_fb=nit_fb, reg_par=lbd, BiographParams=ParamsGPUEM, dict_sino=dict_sino)
x_admm = x_admm.squeeze().cpu().detach().numpy()

fig=PETLibs.utils.display3D_1cbar(x_admm,interpolation='none', perc=0.7, title='reconstruction with FB - TV -l2 regularization')

Construct and load GSDenoiser

In [ ]:

from DL_utils import MyDeepModel2


class MyGSDenoiser(torch.nn.Module):
    def __init__(self):
        super(MyGSDenoiser, self).__init__()
        self.base = MyDeepModel2()

    def forward(self, x):
        x = x.float()
        x = x.requires_grad_()
        N = self.base.forward(x)
        JN = torch.autograd.grad(N, x, grad_outputs=x - N, create_graph=True, only_inputs=True)[0]
        Dg = x - N - JN
        x_hat = x - Dg

        x_hat = x_hat.detach()
        x = x.detach()
        N = N.detach()
        JN = JN.detach()


        return x_hat

    def getN(self, x):
        x = x.float()
        # x = x.requires_grad_()
        N = self.base.forward(x)
        return N



GSnet = MyGSDenoiser()
GSnet = GSnet.cuda()


filename = 'gsnet.ckpt'
checkpoint = torch.load(filename)
GSnet.load_state_dict(checkpoint['state_dict'], strict=True)
GSnet.eval()


Write the PnP version of the forward backward algorithm with GSnet

In [ ]:


def pnp_fb(x0, nit_fb, reg_par, BiographParams, dict_sino, net):

    x=torch.tensor(np.zeros_like(x0), dtype=torch.float32,device="cuda")



    tau = 1.99/(reg_par)
    epsilon = 1e-9
    delta = epsilon + 1e-6
    gamma = 1e-5
    eta = 0.9
    i=0
    xprec = x


    while i <nit_fb and delta > epsilon:
        print("#########################################################")
        print(f"i={i} over {nit_fb}")
        print("#########################################################")

        xprec = x
        backtracking_check = False
        sub_counter = 0


        while not backtracking_check:
            # print("#########################################################")
            # print(x.shape)
            # print(preprocess_data(x, tensorFlag=True).shape)
            # print(net(preprocess_data(x, tensorFlag=True)).shape)
            u =  postprocess_data((tau*reg_par*net(preprocess_data(x, tensorFlag=True))),tensorFlag=True) + (1-reg_par*tau)*x
            # print(u.shape)


            xprox=u
            x=GPUBiographProxRecons(BiographReconsParams=BiographParams,xprox=xprox,pnlt_beta=1.0/tau,dict_sino=dict_sino, tensor_output = True).squeeze()
            # print(x.shape)

            lk = GPU_log_likelihood(x, eps =0, BiographParams=BiographParams,dict_sino=dict_sino)
            # x=torch.swapaxes(x,2,0).cpu()


            ## compute regularization
            Nx = postprocess_data(net.getN(preprocess_data(x,tensorFlag=True)),tensorFlag=True)
            gx = 0.5*torch.sum((x - Nx).flatten() ** 2).item()

            if sub_counter > 0:
                lossprec = loss
            if i== 0 and sub_counter == 0:
                linit = lk

            loss = -lk + gx


            ## Update parameters
            if (sub_counter > 0 and np.abs(lossprec -lk) < (gamma/tau)*torch.norm(xprec-x)**2) :
                tau = eta*tau
                x = xprec
                backtracking_check = False
            else:
                if sub_counter > 0 and i > 0:
                    delta  = np.abs((lossprec-lk)/linit)
                backtracking_check = True


            sub_counter += 1
            i += 1

    return x

Test PnP reconstruction with GSnet

In [ ]:
nit_fb = 20
lbd = 0.00002

x_admm = pnp_fb(x0=ordose, nit_fb=nit_fb, reg_par=lbd, BiographParams=ParamsGPUEM, dict_sino=dict_sino, net=GSnet)
x_admm = x_admm.squeeze().cpu().detach().numpy()



In [ ]:
fig=PETLibs.utils.display3D_1cbar(x_admm,interpolation='none', perc=0.7, title='reconstruction with PnP FB')